In [2]:
import pandas as pd
import numpy as np
import copy
import random

from games4e import Game, GameState, minmax_decision, alpha_beta_cutoff_search


In [3]:
class Mancala(Game):
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones
        self.p1_pits_index = [0, pits_per_player-1]
        self.p1_mancala_index = pits_per_player
        self.p2_pits_index = [pits_per_player+1, len(board)-2] 
        self.p2_mancala_index = len(board)-1
        
        # Zero out the Mancala pits
        board[self.p1_mancala_index] = 0
        board[self.p2_mancala_index] = 0
        
        # Create initial GameState
        moves = list(range(self.p1_pits_index[0], self.p1_pits_index[1]+1))
        self.initial = GameState(to_move=1, utility=0, board=board, moves=moves)

    def actions(self, state: GameState) -> list[int]:
        # GameState = namedtuple('GameState', 'to_move, utility, board, moves')
        # available actions based on the player to move and the board state
        if state.to_move == 1:
            return [pit for pit in range(self.p1_pits_index[0], self.p1_pits_index[1]+1) if state.board[pit] > 0]
        else:
            return [pit for pit in range(self.p2_pits_index[0], self.p2_pits_index[1]+1) if state.board[pit] > 0]
    
    def result(self, state: GameState, move: int) -> GameState:
        # update the board state after a move is made
        new_state = copy.deepcopy(state)
        board = new_state.board
        current_player = state.to_move
        
        # Convert move to board index if player 2
        pit = move if current_player == 1 else move
        
        # Get stones from pit
        stones = board[pit]
        board[pit] = 0
        
        # Distribute stones
        current_pit = pit
        while stones > 0:
            current_pit = (current_pit + 1) % len(board)
            # Skip opponent's mancala
            if current_player == 1 and current_pit == self.p2_mancala_index:
                continue
            if current_player == 2 and current_pit == self.p1_mancala_index:
                continue
            board[current_pit] += 1
            stones -= 1
            
        # Handle capture
        # Check if the last stone lands in an empty pit
        if board[current_pit] == 1:
            # Check if the last stone lands in the player's own pits
            if current_player == 1 and current_pit in range(self.p1_pits_index[0], self.p1_pits_index[1]+1):
                # Get the opposite pit index
                opposite_pit = self.p2_pits_index[1] - current_pit
                # Check if the opposite pit is not empty
                if board[opposite_pit] != 0:
                    # Capture the stones in the opposite pit and the last stone
                    board[self.p1_mancala_index] += board[opposite_pit] + 1
                    board[opposite_pit] = 0
                    board[current_pit] = 0
                    
            # Check if the last stone lands in the player's own pits
            if current_player == 2 and current_pit in range(self.p2_pits_index[0], self.p2_pits_index[1]+1):
                # Get the opposite pit index
                opposite_pit = self.p1_pits_index[1] - (current_pit - 7)
                # Check if the opposite pit is not empty
                if board[opposite_pit] != 0:
                    # Capture the stones in the opposite pit and the last stone
                    board[self.p2_mancala_index] += board[opposite_pit] + 1
                    board[opposite_pit] = 0
                    board[current_pit] = 0
        # Switch player
        next_player = 1 if current_player == 2 else 2
        
        # Get available moves for next player
        if next_player == 1:
            moves = [pit for pit in range(self.p1_pits_index[0], self.p1_pits_index[1]+1) if board[pit] > 0]
        else:
            moves = [pit for pit in range(self.p2_pits_index[0], self.p2_pits_index[1]+1) if board[pit] > 0]
            
        return GameState(to_move=next_player, utility=0, board=board, moves=moves)
        
    def utility(self, state: GameState, player: int) -> int:
        # return the utility of the game state for the given player
        utility = state.board[self.p1_mancala_index] - state.board[self.p2_mancala_index]
        return utility if player == 1 else -utility
    
    def terminal_test(self, state: GameState) -> bool:
        # check if the game is in a terminal state
        for pit in range(self.p1_pits_index[0], self.p1_pits_index[1]+1):
            if state.board[pit] != 0:
                return False
        for pit in range(self.p2_pits_index[0], self.p2_pits_index[1]+1):
            if state.board[pit] != 0:
                return False
        return True
    
    def to_move(self, state: GameState) -> int:
        # return the player whose turn it is
        return state.to_move    
    
    def display(self, state: GameState):
        # print the game state
        player_1_pits = state.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = state.board[self.p1_mancala_index]
        player_2_pits = state.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = state.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if state.to_move == 1 else 'P2'
        print('Turn: ' + turn)
    
    def play(self, player1, player2):
        state = self.initial
        moves = 0

        while True:
            if self.terminal_test(state):
                break

            moves += 1
            available_actions = self.actions(state)
            if not available_actions:
                break

            if state.to_move == 1:
                action = player1(self, state)
            else:
                action = player2(self, state)

            if action is None: 
                break

            state = self.result(state, action)

        if state.board[self.p1_mancala_index] > state.board[self.p2_mancala_index]:
            winner = 1 
        elif state.board[self.p1_mancala_index] < state.board[self.p2_mancala_index]:
            winner = 2
        else:
            winner = 0  # Draw

        return winner, moves

    def simulate(self, player1, player2, num_games):
        results = [0, 0, 0, 0]  # player1 wins, player2 wins, draws, total moves
        for i in range(num_games):
            winner, moves = self.play(player1, player2)
            results[3] += moves
            if winner == 1:
                results[0] += 1
            elif winner == 2:
                results[1] += 1
            else:
                results[2] += 1
        return results


In [4]:
def minmax_decision(state, game, d=5):  
    """Given a state in a game, calculate the best move by searching
    forward to a specified depth or terminal states."""

    player = game.to_move(state)

    def max_value(state, depth):  
        if depth == 0 or game.terminal_test(state):  # Check depth
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), depth - 1))  # Decrease depth
        return v

    def min_value(state, depth): 
        if depth == 0 or game.terminal_test(state):  # Check depth
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), depth - 1))  # Decrease depth
        return v

    # Body of minmax_decision:
    return max(game.actions(state), 
              key=lambda a: min_value(game.result(state, a), d-1))  # Start depth search

In [5]:
game = Mancala()

def random_player(game, state):
    return random.choice(game.actions(state))

def minmax_player(game, state, depth):
    actions = game.actions(state)
    if not actions: 
        return None
    return minmax_decision(state, game, d=depth)

def ab_player(game, state, depth):
    actions = game.actions(state)
    if not actions: 
        return None
    return alpha_beta_cutoff_search(state, game, d=depth)


In [6]:
test1 = game.simulate(random_player, random_player, 100)
print("100 trials of random p1 vs random p2:")
print("Random 1 Wins:", test1[0], "\nRandom 2 Wins:", test1[1], "\nDraws:", test1[2], "\nAverage Moves in a game:", test1[3] / 100)

100 trials of random p1 vs random p2:
Random 1 Wins: 54 
Random 2 Wins: 43 
Draws: 3 
Average Moves in a game: 48.03


## 3. Play 100 games of random player against random player

### 3.1. What percentage of games does each player (1st or 2nd) win?

- The first player wins a little bit more, 53%, while the second player wins 47% of the time.  

### 3.2. On average, how many moves does it take to win?

- For random vs random, it took an average of 46 moves to complete a game.

In [7]:
smaller_game = Mancala(pits_per_player=3, stones_per_pit=2)
print("\n100 trials of random p1 vs minmax ai")
test2 = smaller_game.simulate(random_player, (lambda g, s: minmax_player(g, s, 5)), 100)
print("Random Wins:", test2[0], "\nMinMax Wins:", test2[1], "\nDraws:", test2[2], "\nAverage Moves in a game:", test2[3] / 100)


100 trials of random p1 vs minmax ai
Random Wins: 25 
MinMax Wins: 35 
Draws: 40 
Average Moves in a game: 11.71


## 4 & 5. Build an AI player that uses minimax to choose the best move with a variable number of plies and a utility function we describe. Play 100 games with the random player against the minimax AI player at a depth of 5 plies


### 5.1. What percentage of games does each player (AI or random) win?

- For random vs minmax, the AI won 30 of the 100 games, the random player won 18 games and 52 games were draws. 

### 5.2. On average, how many moves does it take to win?

- It took an average of 11.24 moves to complete a game. The game was played on a smaller board with 3 pits per player and 2 stones per pit.

### 5.3. Is your AI player better than random chance? Write a paragraph or two describing or why not.

Yes, the AI player is better than random chance. We can see that consistantly the AI player wins more often than the random player. Additionally, when it was the two random players playing against each other it took more moves to complete a game. In the first test, the two random players won at a chance closer to 50-50. We can see a contrast here with player 2 winning significantly more often over 100 seperate trials. Even when the AI player did not win, it managed to draw more games than the random player. 

In [8]:
print("\n100 trials of random p1 vs ab ai")
test3 = game.simulate(random_player, (lambda g, s: ab_player(g, s, 5)), 100)
print("Random Wins:", test3[0], "\nAB Wins:", test3[1], "\nDraws:", test3[2], "\nAverage Moves in a game:", test3[3] / 100)


100 trials of random p1 vs ab ai
Random Wins: 1 
AB Wins: 99 
Draws: 0 
Average Moves in a game: 32.84


In [9]:
print("\n1 trials of random p1 vs ab ai")
test4 = game.simulate(random_player, (lambda g, s: ab_player(g, s, 5)), 1)
print("Random Wins:", test4[0], "\nAB Wins:", test4[1], "\nDraws:", test4[2], "\nAverage Moves in a game:", test4[3] / 1)



1 trials of random p1 vs ab ai
Random Wins: 0 
AB Wins: 1 
Draws: 0 
Average Moves in a game: 34.0


## 7. Play 100 games with the random player against the Alpha-Beta AI player at a
depth of 5 plies

### 7.1. How long does it take for a single game to run to completion?

For a single game to run, it takes about 0.1 seconds to run, and to run all 100 trials it takes roughly 24 seconds to run.

### 7.2. What percentage of games does each player (AI or random) win?

The AI using AB pruning takes the win 99 of the 100 games, while the random player wins 1 game.

### 7.3. On average, how many moves does it take to win?

It takes 33 moves to win a game on average.

### 7.4. Are your results for this part different from those for your minimax AI player?


Compared to the minimax AI player, the AB pruning AI player wins more games and takes less time to complete a game. The AB pruning AI is much more consistent in beating the random player compared to the minimax AI, however, this might be due to the AB AI being played on the full board.


In [18]:
print("\n100 trials of random p1 vs ab ai")
test3 = smaller_game.simulate(random_player, (lambda g, s: ab_player(g, s, 10)), 100)
print("Random Wins:", test3[0], "\nAB Wins:", test3[1], "\nDraws:", test3[2], "\nAverage Moves in a game:", test3[3] / 100)




100 trials of random p1 vs ab ai
Random Wins: 64 
AB Wins: 9 
Draws: 27 
Average Moves in a game: 6.82


In [14]:
print("\n1 trial of random p1 vs ab ai (full board)")
test4 = game.simulate(random_player, (lambda g, s: ab_player(g, s, 10)), 1)
print("Random Wins:", test4[0], "\nAB Wins:", test4[1], "\nDraws:", test4[2], "\nAverage Moves in a game:", test4[3] / 1)


1 trial of random p1 vs ab ai (full board)
Random Wins: 0 
AB Wins: 1 
Draws: 0 
Average Moves in a game: 33.0


In [20]:
medium_game = Mancala(pits_per_player=5, stones_per_pit=3)
print("\n100 trials of random p1 vs ab ai (medium board)")
test5 = medium_game.simulate(random_player, (lambda g, s: ab_player(g, s, 10)), 100)
print("Random Wins:", test5[0], "\nAB Wins:", test5[1], "\nDraws:", test5[2], "\nAverage Moves in a game:", test5[3] / 100)



100 trials of random p1 vs ab ai (medium board)
Random Wins: 30 
AB Wins: 63 
Draws: 7 
Average Moves in a game: 18.72


In [19]:
print("\n100 trial of random p1 vs ab ai (medium board, depth=5)")
test6 = medium_game.simulate(random_player, (lambda g, s: ab_player(g, s, 5)), 100)
print("Random Wins:", test6[0], "\nAB Wins:", test6[1], "\nDraws:", test6[2], "\nAverage Moves in a game:", test6[3] / 100)


100 trial of random p1 vs ab ai (medium board, depth=5)
Random Wins: 1 
AB Wins: 97 
Draws: 2 
Average Moves in a game: 23.92


## 8. (Extra Credit, 10 points). Play 100 games with the random player against the
Alpha-Beta AI player at a depth of 10 plies

### 8.1 How long does it take for a single game to run to completion?

- It takes about 32s to run a full mancala game (6 pits, 4 stones per pit)
- It takes about 0.3s to run 100 smaller mancala game (3 pits, 2 stones per pit)

### 8.2 What percentage of games does each player (AI or random) win?

- In the smaller game, the AI player won only 9 games, while the random won 64 games.
- In the medium game (5 pits, 3 stones per pit), the AI player won 61 games, while the random won 32 games.

### 8.3 On average, how many moves does it take to win?

- In the smaller game, it took an average of 6.82 moves to win a game.
- In the medium game, it took an average of 20.41 moves to win a game.

### 8.4 Does increasing the number of plies improve the play for our AI player? Why or why not?

- No it surprisingly did not improve the performance of the AI player. This might be due to our limited utility function that is relatively simple by only accounting for number of stones in the mancala pits. 